In [26]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F  # useful stateless functions
import numpy as np

In [27]:
def loadDataAsTensor():
    # Assume we only deal with 6 possible notes: C, D, E, F, G, and _
    # correspond to a one hot array with value 1 at indexes 0, 1, 2, 3, 4, 5
    # Tempo consists of 1, 2, 3, 4

    k = torch.LongTensor([[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0], [0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1]])
    p1 = torch.LongTensor([0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3])
    p2 = torch.LongTensor([4, 5, 5, 5, 4, 5, 5, 5, 4, 5, 5, 5])
    p3 = torch.LongTensor([5, 2, 5, 2, 5, 2, 5, 2, 5, 2, 5, 2])
    p4 = torch.LongTensor([0, 5, 0, 5, 0, 5, 0, 5, 0, 5, 0, 5])
    tempo = torch.LongTensor([1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4])
    
    # Turn notes into onehot numpy vectors
    x1 = k[p1]
    x2 = k[p2]
    x3 = k[p3]
    x4 = k[p4]
    
    return x1, x2, x3, x4, tempo

# loadDataAsTensor()


In [28]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.tanh = nn.Tanh()
        self.h2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax()
        
    def forward(self, x2, x3, x4, tempo, hidden):
        temp = torch.cat((x2[0].float(), x3[0].float(), x4[0].float(), tempo.float(), hidden[0].float()), 0)
        input_combined = temp.view(1, -1).float()
        
        hidden_i = self.i2h(input_combined)
        hidden_i = self.tanh(hidden_i)

        output = self.h2o(hidden_i)
        output = self.softmax(output)
        return output, hidden_i
    
    def initHidden(self):
        return torch.zeros(1, self.hidden_size, dtype=torch.long)

In [29]:
# Assume we are predicting p1. Hence the training data includes p2, p3, p4, and tempo

def getTrainingDataTensor(idx, x1, x2, x3, x4, tempo):
    x1_tensor = x1[idx, :].view(1, -1)
    x2_tensor = x2[idx, :].view(1, -1)
    x3_tensor = x3[idx, :].view(1, -1)
    x4_tensor = x4[idx, :].view(1, -1)
    tempo_tensor = torch.zeros(1).long()
    tempo_tensor.fill_(tempo[idx]).view(1, -1)
    
    return x2_tensor, x3_tensor, x4_tensor, tempo_tensor

In [30]:
def getTargetDataTensor(idx, x1):
    x1_tensor = x1[idx, :]
    
    correct_index_as_tensor = (x1_tensor==1).nonzero()[0]
    
    return correct_index_as_tensor

In [31]:
def train(x2_tensor, x3_tensor, x4_tensor, tempo_tensor, target_tensor, hidden_tensor, optimizer):
        
    output, hidden_i = rnn(x2_tensor, x3_tensor, x4_tensor, tempo_tensor, hidden_tensor)

    loss = F.cross_entropy(output, target_tensor)
    optimizer.zero_grad()
    loss.backward(retain_graph=True)
    optimizer.step()
        
    return output, hidden_i, loss.item()


In [32]:
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [33]:
input_size = 6*3+1
hidden_size = 100
output_size = 6
sequence_size = 12

rnn = RNN(input_size, hidden_size, output_size)

n_iters = 2000
print_every = 100
plot_every = 500
all_losses = []
total_loss = 0 # Reset every plot_every iters

start = time.time()
hidden = rnn.initHidden()
print(hidden.type())

tx1, tx2, tx3, tx4, t_tempo = loadDataAsTensor()

learning_rate = 0.005

optimizer = optim.SGD(rnn.parameters(), lr=learning_rate)

ite = 0
for n in range(n_iters):
    if ite/12 >= 1:
        ite = 0
            
    x2, x3, x4, tempo = getTrainingDataTensor(ite, tx1, tx2, tx3, tx4, t_tempo)
    target = getTargetDataTensor(ite, tx1)

    output, hidden, loss = train(x2, x3, x4, tempo, target, hidden, optimizer)
    total_loss += loss
    
    if (n+1) % print_every == 0:
            print('%s (%d %d%%) loss: %.4f' % (timeSince(start), n, n / n_iters * 100, loss))
            print("output: {x}".format(x=output))
            
    if (n+1) % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0
    
    ite += 1
    n += 1

torch.LongTensor


/anaconda3/envs/cs682project/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0m 0s (99 4%) loss: 1.7567
output: tensor([[0.1323, 0.1915, 0.1700, 0.2020, 0.1465, 0.1577]], grad_fn=<SoftmaxBackward>)
0m 1s (199 9%) loss: 1.7129
output: tensor([[0.1300, 0.1987, 0.1791, 0.2466, 0.1178, 0.1278]], grad_fn=<SoftmaxBackward>)
0m 3s (299 14%) loss: 1.6597
output: tensor([[0.1249, 0.1998, 0.1815, 0.3013, 0.0920, 0.1004]], grad_fn=<SoftmaxBackward>)
0m 6s (399 19%) loss: 1.5988
output: tensor([[0.1175, 0.1949, 0.1761, 0.3648, 0.0701, 0.0766]], grad_fn=<SoftmaxBackward>)
0m 10s (499 24%) loss: 1.5363
output: tensor([[0.1090, 0.1864, 0.1637, 0.4308, 0.0528, 0.0573]], grad_fn=<SoftmaxBackward>)
0m 14s (599 29%) loss: 1.4801
output: tensor([[0.0999, 0.1794, 0.1469, 0.4909, 0.0400, 0.0429]], grad_fn=<SoftmaxBackward>)
0m 19s (699 34%) loss: 1.4314
output: tensor([[0.0864, 0.1798, 0.1269, 0.5440, 0.0306, 0.0323]], grad_fn=<SoftmaxBackward>)
0m 24s (799 39%) loss: 1.3816
output: tensor([[0.0603, 0.1919, 0.1014, 0.5992, 0.0233, 0.0238]], grad_fn=<SoftmaxBackward>)
0m 30s (899 44%

In [34]:
def convertToNote(idx):
    if idx == 0:
        return 'C'
    elif idx == 1:
        return 'D'
    elif idx == 2:
        return 'E'
    elif idx == 3:
        return 'F'
    elif idx == 4:
        return 'G'
    elif idx == '5':
        return "_"

In [35]:
def sample(length):
    tx1, tx2, tx3, tx4, t_tempo = loadDataAsTensor()
    print(t_tempo)

    with torch.no_grad():
        hidden = rnn.initHidden()
        
        k = 0
        for i in range(length):
            if k / 12 >= 1:
                k = 0
            x2, x3, x4, tempo = getTrainingDataTensor(k, tx1, tx2, tx3, tx4, t_tempo)
            output, hidden = rnn(x2, x3, x4, tempo, hidden)
            
            result = np.random.choice(range(6), p=output.numpy()[0])
            print(convertToNote(result))
            
            k += 1
            i += 1
            
    

In [40]:
sample(20)

tensor([1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4])
E
D
F
F
C
D
E
F
C
D
E
F
C
D
E
F
C
D
E
F


/anaconda3/envs/cs682project/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
